In [8]:
import tensorflow as tf
tf.__version__


'2.12.0'

In [9]:
import numpy as np
import pandas as pd
from sklearn.metrics import mean_absolute_error
import random

In [10]:
subject = 'São Paulo - Consumo de Cimento (t)'
split_index = 6 #Referente aos 7 anos de input  

In [11]:
data = pd.read_csv('../Inputs Atuais/2014_01_model_input_SP.csv')
data = data[[col for col in data.columns if col != subject] + [subject]] #Seta consumo (target) para a coluna final
data

,Ano,São Paulo - IDH Longevitude,São Paulo - IDH,São Paulo - PIB - Estadual,São Paulo - PIB - Construção Civil,São Paulo - PIB - Per Capita,São Paulo - PIB - Preços de Mercado,Contratação Comercial (Bi R$),Contratação Home equity (Bi R$),Contratação Livre (Bi R$),...,Atmospheric Pressure(mB),Global Radiation - Daylight (Kj/m²),Air Temperature - Dry Bulb (°C),Dew Point Temperature (°C),Air Relative Humidity (%),Wind Gust (m/s),São Paulo - Desemprego,São Paulo - IDH Renda,São Paulo - IDH Educacao,São Paulo - Consumo de Cimento (t)
0,2014,0.849730,0.821077,1.197826e+09,4.735435e+07,24.001434,1.003012e+09,5.676413e+08,2.779340e+09,1.954361e+10,...,836.986331,1108.545094,15.247907,11.986366,82.848722,1.532951,11.415800,0.793855,0.821991,13714.511286
1,2015,0.860278,0.824371,1.206909e+09,4.689336e+07,23.951008,1.001922e+09,5.179392e+08,2.561731e+09,1.327596e+10,...,837.040042,1132.416417,15.520197,12.242333,82.064855,1.539659,11.865646,0.784967,0.830674,11767.653429
2,2016,0.858485,0.824495,1.212819e+09,4.650979e+07,23.922375,1.001305e+09,4.524144e+08,2.516859e+09,1.011370e+10,...,837.011044,1181.481000,15.455320,11.972800,81.279861,1.362167,11.874466,0.785237,0.832462,9811.084286
3,2017,0.856075,0.823540,1.214213e+09,4.632537e+07,23.910733,1.000929e+09,4.213689e+08,2.497710e+09,8.456309e+09,...,837.006952,1198.617570,15.304181,12.004953,82.751052,1.502384,11.486716,0.785077,0.832050,9918.501000
4,2018,0.853082,0.822862,1.211766e+09,4.629030e+07,23.902108,1.000151e+09,4.166580e+08,2.797145e+09,7.937760e+09,...,836.989876,1222.116471,15.275935,12.258430,83.908178,1.685017,11.319293,0.785447,0.832600,10001.077000
5,2019,0.849362,0.821577,1.204757e+09,4.635890e+07,23.895877,9.992364e+08,4.462625e+08,3.484252e+09,1.091282e+10,...,836.905187,1236.895592,15.118043,12.137404,83.369220,1.479098,11.198876,0.784943,0.832927,10458.276000
6,2020,0.844940,0.819448,1.190468e+09,4.655413e+07,23.874352,9.969671e+08,5.489412e+08,4.114911e+09,1.625333e+10,...,836.629000,1193.582762,15.075998,11.537222,80.653205,1.231389,11.044481,0.784392,0.831466,11181.809000
7,2021,0.845026,0.819125,1.171137e+09,4.681715e+07,23.903240,9.959823e+08,6.638811e+08,4.421848e+09,1.933031e+10,...,836.382749,1074.106519,14.961645,11.014166,79.055593,1.296973,10.845149,0.784700,0.830054,12113.838000
8,2022,0.848210,0.820318,1.147642e+09,4.713421e+07,23.936645,9.946419e+08,7.010332e+08,4.533216e+09,1.997888e+10,...,836.362997,988.639283,14.788160,10.955363,79.586575,1.316356,10.854380,0.785946,0.829241,12383.649000


In [12]:
input_data = data.iloc[:-2, 1:-1]
input_data

,São Paulo - IDH Longevitude,São Paulo - IDH,São Paulo - PIB - Estadual,São Paulo - PIB - Construção Civil,São Paulo - PIB - Per Capita,São Paulo - PIB - Preços de Mercado,Contratação Comercial (Bi R$),Contratação Home equity (Bi R$),Contratação Livre (Bi R$),Contratação FGTS (Bi R$),...,Precipitation (mm/day),Atmospheric Pressure(mB),Global Radiation - Daylight (Kj/m²),Air Temperature - Dry Bulb (°C),Dew Point Temperature (°C),Air Relative Humidity (%),Wind Gust (m/s),São Paulo - Desemprego,São Paulo - IDH Renda,São Paulo - IDH Educacao
0,0.849730,0.821077,1.197826e+09,4.735435e+07,24.001434,1.003012e+09,5.676413e+08,2.779340e+09,1.954361e+10,4.143397e+10,...,15.750987,836.986331,1108.545094,15.247907,11.986366,82.848722,1.532951,11.415800,0.793855,0.821991
1,0.860278,0.824371,1.206909e+09,4.689336e+07,23.951008,1.001922e+09,5.179392e+08,2.561731e+09,1.327596e+10,4.469706e+10,...,15.893940,837.040042,1132.416417,15.520197,12.242333,82.064855,1.539659,11.865646,0.784967,0.830674
2,0.858485,0.824495,1.212819e+09,4.650979e+07,23.922375,1.001305e+09,4.524144e+08,2.516859e+09,1.011370e+10,4.640544e+10,...,15.955560,837.011044,1181.481000,15.455320,11.972800,81.279861,1.362167,11.874466,0.785237,0.832462
3,0.856075,0.823540,1.214213e+09,4.632537e+07,23.910733,1.000929e+09,4.213689e+08,2.497710e+09,8.456309e+09,4.607888e+10,...,16.984321,837.006952,1198.617570,15.304181,12.004953,82.751052,1.502384,11.486716,0.785077,0.832050
4,0.853082,0.822862,1.211766e+09,4.629030e+07,23.902108,1.000151e+09,4.166580e+08,2.797145e+09,7.937760e+09,4.467878e+10,...,16.610359,836.989876,1222.116471,15.275935,12.258430,83.908178,1.685017,11.319293,0.785447,0.832600
5,0.849362,0.821577,1.204757e+09,4.635890e+07,23.895877,9.992364e+08,4.462625e+08,3.484252e+09,1.091282e+10,4.379096e+10,...,14.690772,836.905187,1236.895592,15.118043,12.137404,83.369220,1.479098,11.198876,0.784943,0.832927
6,0.844940,0.819448,1.190468e+09,4.655413e+07,23.874352,9.969671e+08,5.489412e+08,4.114911e+09,1.625333e+10,4.320994e+10,...,-402.031586,836.629000,1193.582762,15.075998,11.537222,80.653205,1.231389,11.044481,0.784392,0.831466


In [13]:
# Shift para prever futuro e não presente
target_data = data[subject].shift(-1)
target_data

0    11767.653429
1     9811.084286
2     9918.501000
3    10001.077000
4    10458.276000
5    11181.809000
6    12113.838000
7    12383.649000
8             NaN
Name: São Paulo - Consumo de Cimento (t), dtype: float64

In [14]:
# input para treinamento
train_input = (input_data.iloc[:-1] - np.mean(input_data.iloc[:-1], axis=0)) / np.std(input_data.iloc[:-1], axis=0)
train_input

,São Paulo - IDH Longevitude,São Paulo - IDH,São Paulo - PIB - Estadual,São Paulo - PIB - Construção Civil,São Paulo - PIB - Per Capita,São Paulo - PIB - Preços de Mercado,Contratação Comercial (Bi R$),Contratação Home equity (Bi R$),Contratação Livre (Bi R$),Contratação FGTS (Bi R$),...,Precipitation (mm/day),Atmospheric Pressure(mB),Global Radiation - Daylight (Kj/m²),Air Temperature - Dry Bulb (°C),Dew Point Temperature (°C),Air Relative Humidity (%),Wind Gust (m/s),São Paulo - Desemprego,São Paulo - IDH Renda,São Paulo - IDH Educacao
0,-1.152229,-1.468721,-1.810772,1.901702,1.950711,1.587258,1.779974,0.019134,2.003355,-1.880579,...,-0.318519,-0.085680,-1.551570,-0.542931,-0.958575,0.170333,0.168445,-0.429787,2.232961,-2.196862
1,1.394689,1.063919,-0.201893,0.704617,0.562225,0.685511,0.870373,-0.621387,0.401153,0.111653,...,-0.120551,1.201860,-1.033316,1.500207,1.193456,-0.750017,0.238748,1.312004,-0.497880,0.057953
2,0.961790,1.159735,0.845075,-0.291420,-0.226178,0.175749,-0.328802,-0.753466,-0.407217,1.154682,...,-0.035216,0.506731,0.031892,1.013400,-1.072631,-1.671692,-1.621515,1.346155,-0.414999,0.522409
3,0.379785,0.425421,1.092030,-0.770296,-0.546738,-0.135236,-0.896969,-0.809829,-0.830898,0.955301,...,1.389464,0.408652,0.403932,-0.120677,-0.802306,0.055657,-0.151926,-0.155203,-0.464242,0.415288
4,-0.342932,-0.096036,0.658588,-0.861368,-0.784231,-0.778570,-0.983185,0.071542,-0.963455,0.100493,...,0.871582,-0.000697,0.914101,-0.332623,1.328788,1.414259,1.762225,-0.803459,-0.350372,0.558113
5,-1.241102,-1.084317,-0.583029,-0.683234,-0.955789,-1.534712,-0.441390,2.094006,-0.202939,-0.441549,...,-1.786759,-2.030866,1.234961,-1.517375,0.311268,0.781459,-0.395977,-1.269710,-0.505467,0.643099


In [15]:
# Alvo para treinamento
train_target = target_data.iloc[:split_index]
train_target

0    11767.653429
1     9811.084286
2     9918.501000
3    10001.077000
4    10458.276000
5    11181.809000
Name: São Paulo - Consumo de Cimento (t), dtype: float64

In [16]:
#input de test (Ano 2021)
test_input = ((input_data - np.mean(input_data, axis=0)) / np.std(input_data, axis=0)).iloc[-1:]
test_input

,São Paulo - IDH Longevitude,São Paulo - IDH,São Paulo - PIB - Estadual,São Paulo - PIB - Construção Civil,São Paulo - PIB - Per Capita,São Paulo - PIB - Preços de Mercado,Contratação Comercial (Bi R$),Contratação Home equity (Bi R$),Contratação Livre (Bi R$),Contratação FGTS (Bi R$),...,Precipitation (mm/day),Atmospheric Pressure(mB),Global Radiation - Daylight (Kj/m²),Air Temperature - Dry Bulb (°C),Dew Point Temperature (°C),Air Relative Humidity (%),Wind Gust (m/s),São Paulo - Desemprego,São Paulo - IDH Renda,São Paulo - IDH Educacao
6,-1.610507,-1.756374,-1.86678,-0.16282,-1.237276,-1.93551,1.16956,2.035192,0.985175,-0.705937,...,-2.449464,-2.342406,0.271094,-1.394884,-2.138289,-1.648541,-1.835018,-1.412546,-0.605246,0.242895


In [17]:
# Alvo de test (Ano 2021)
test_target = target_data.iloc[split_index:-2]
test_target

6    12113.838
Name: São Paulo - Consumo de Cimento (t), dtype: float64

In [18]:
# Rede neural feed-forward com optmizador Estocástico
def neural_network_model(train_input, train_target, want_verbose=1, seed=0):
    if seed != 0:
        random.seed(seed)
        np.random.seed(seed)
        tf.random.set_seed(seed)
    # Aṕos 500 epochs sem grandes melhoras no val_loss, interrompe.
    early_stopping = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=500, verbose=1)
    # Método estocástico e learning rate=0.01
    optimizer = tf.keras.optimizers.SGD(learning_rate=0.01)
    model = tf.keras.Sequential([
        tf.keras.layers.Dense(2048, activation='tanh', input_shape=(train_input.shape[1],)),
        tf.keras.layers.Dropout(0.5),
        tf.keras.layers.Dense(2048, activation='tanh'),
        tf.keras.layers.Dropout(0.3),
        tf.keras.layers.Dense(1024, activation='tanh'),
        tf.keras.layers.Dense(512, activation='tanh'),
        tf.keras.layers.Dense(64, activation='tanh'),
        tf.keras.layers.Dense(1)
    ])
    model.compile(optimizer=optimizer, loss='mean_squared_error')   
    history = model.fit(train_input, 
                      train_target, 
                      epochs=5000, 
                      batch_size=len(train_input), 
                      validation_data=(train_input.iloc[-1:], train_target.iloc[-1:]),
                      callbacks=[early_stopping], 
                      verbose=want_verbose)
    return model, history

In [19]:
def get_a_good_seed(train_input, train_target, test_target, test_input):

    random_seeds = [random.randint(0, 2**32 - 1) for _ in range(10)]
    print(random_seeds)

    best_val_loss = float('inf')
    winner_seed = None
    i = 0
    for seed in random_seeds:
        print(f"\n\nStep: {i} ___________________________________________")
        i += 1

        model, history = neural_network_model(train_input, train_target, 0, seed)

        current_val_loss = min(history.history['val_loss'])
        print(f"val_loss: {current_val_loss}")

        if current_val_loss < best_val_loss:
            best_val_loss = current_val_loss
            winner_seed = seed
            print(f"winner_seed: {winner_seed}")

    return winner_seed

In [28]:
winner_seed = get_a_good_seed(train_input, train_target, test_target, test_input)
print(f"\n\nfinal_seed: {winner_seed}")

[2858971814, 51309233, 2763338100, 1439685192, 1165178730, 1129759923, 4127240961, 2743300736, 34086500, 1591734534]


Step: 0 ___________________________________________
Epoch 1011: early stopping
val_loss: 1825.8929443359375
winner_seed: 2858971814


Step: 1 ___________________________________________
Epoch 1435: early stopping
val_loss: 13904.4169921875


Step: 2 ___________________________________________
Epoch 855: early stopping
val_loss: 0.007897377014160156
winner_seed: 2763338100


Step: 3 ___________________________________________
Epoch 610: early stopping
val_loss: 1790.5955810546875


Step: 4 ___________________________________________
Epoch 526: early stopping
val_loss: 5.566650390625


Step: 5 ___________________________________________
Epoch 1035: early stopping
val_loss: 0.39429569244384766


Step: 6 ___________________________________________
Epoch 789: early stopping
val_loss: 12562.1630859375


Step: 7 ___________________________________________
Epoch 871: early sto

In [29]:
trained_model, history = neural_network_model(train_input, train_target, want_verbose=1, seed=winner_seed)

Epoch 1/5000
1/1 [==============================] - 0s 322ms/step - loss: 111259320.0000 - val_loss: 116797032.0000
Epoch 2/5000
1/1 [==============================] - 0s 27ms/step - loss: 112248344.0000 - val_loss: 142918592.0000
Epoch 3/5000
1/1 [==============================] - 0s 41ms/step - loss: 117548968.0000 - val_loss: 126345920.0000
Epoch 4/5000
1/1 [==============================] - 0s 32ms/step - loss: 85772392.0000 - val_loss: 88435624.0000
Epoch 5/5000
1/1 [==============================] - 0s 29ms/step - loss: 81107312.0000 - val_loss: 73343872.0000
Epoch 6/5000
1/1 [==============================] - 0s 26ms/step - loss: 79538712.0000 - val_loss: 76390448.0000
Epoch 7/5000
1/1 [==============================] - 0s 30ms/step - loss: 62546416.0000 - val_loss: 90511640.0000
Epoch 8/5000
1/1 [==============================] - 0s 30ms/step - loss: 84772024.0000 - val_loss: 33800088.0000
Epoch 9/5000
1/1 [==============================] - 0s 28ms/step - loss: 67788136.0000 - 

Epoch 73/5000
1/1 [==============================] - 0s 30ms/step - loss: 13206461.0000 - val_loss: 7743909.5000
Epoch 74/5000
1/1 [==============================] - 0s 28ms/step - loss: 11427696.0000 - val_loss: 6818519.5000
Epoch 75/5000
1/1 [==============================] - 0s 28ms/step - loss: 8279836.0000 - val_loss: 7211858.0000
Epoch 76/5000
1/1 [==============================] - 0s 34ms/step - loss: 14079735.0000 - val_loss: 7954107.5000
Epoch 77/5000
1/1 [==============================] - 0s 40ms/step - loss: 10864553.0000 - val_loss: 6754181.5000
Epoch 78/5000
1/1 [==============================] - 0s 35ms/step - loss: 7337720.0000 - val_loss: 6851367.5000
Epoch 79/5000
1/1 [==============================] - 0s 29ms/step - loss: 15404092.0000 - val_loss: 7430426.5000
Epoch 80/5000
1/1 [==============================] - 0s 26ms/step - loss: 7398397.5000 - val_loss: 5259914.0000
Epoch 81/5000
1/1 [==============================] - 0s 26ms/step - loss: 9399045.0000 - val_loss: 

1/1 [==============================] - 0s 28ms/step - loss: 877494.4375 - val_loss: 4231253.0000
Epoch 146/5000
1/1 [==============================] - 0s 26ms/step - loss: 2575385.2500 - val_loss: 2467817.0000
Epoch 147/5000
1/1 [==============================] - 0s 26ms/step - loss: 1437384.8750 - val_loss: 3168243.5000
Epoch 148/5000
1/1 [==============================] - 0s 27ms/step - loss: 2065568.0000 - val_loss: 1509219.5000
Epoch 149/5000
1/1 [==============================] - 0s 26ms/step - loss: 827489.0625 - val_loss: 2435943.5000
Epoch 150/5000
1/1 [==============================] - 0s 26ms/step - loss: 1693971.5000 - val_loss: 2568003.0000
Epoch 151/5000
1/1 [==============================] - 0s 29ms/step - loss: 2211078.2500 - val_loss: 3209524.7500
Epoch 152/5000
1/1 [==============================] - 0s 29ms/step - loss: 1454801.6250 - val_loss: 2380415.0000
Epoch 153/5000
1/1 [==============================] - 0s 27ms/step - loss: 4038395.7500 - val_loss: 1619343.2500


1/1 [==============================] - 0s 30ms/step - loss: 706886.6875 - val_loss: 278544.8125
Epoch 219/5000
1/1 [==============================] - 0s 49ms/step - loss: 521952.4062 - val_loss: 306409.8125
Epoch 220/5000
1/1 [==============================] - 0s 36ms/step - loss: 1449386.1250 - val_loss: 354688.8750
Epoch 221/5000
1/1 [==============================] - 0s 29ms/step - loss: 369882.9062 - val_loss: 249046.8125
Epoch 222/5000
1/1 [==============================] - 0s 26ms/step - loss: 556504.6250 - val_loss: 703429.5000
Epoch 223/5000
1/1 [==============================] - 0s 27ms/step - loss: 463664.0000 - val_loss: 739450.5000
Epoch 224/5000
1/1 [==============================] - 0s 28ms/step - loss: 436758.1562 - val_loss: 549636.8750
Epoch 225/5000
1/1 [==============================] - 0s 28ms/step - loss: 316251.6250 - val_loss: 345744.0000
Epoch 226/5000
1/1 [==============================] - 0s 26ms/step - loss: 514565.9062 - val_loss: 296870.6875
Epoch 227/5000


Epoch 292/5000
1/1 [==============================] - 0s 31ms/step - loss: 404956.9062 - val_loss: 91856.9453
Epoch 293/5000
1/1 [==============================] - 0s 37ms/step - loss: 227094.3281 - val_loss: 132968.4844
Epoch 294/5000
1/1 [==============================] - 0s 30ms/step - loss: 695701.4375 - val_loss: 404076.2500
Epoch 295/5000
1/1 [==============================] - 0s 27ms/step - loss: 179493.8750 - val_loss: 200914.9375
Epoch 296/5000
1/1 [==============================] - 0s 26ms/step - loss: 73119.6797 - val_loss: 230093.5469
Epoch 297/5000
1/1 [==============================] - 0s 27ms/step - loss: 263827.6250 - val_loss: 154019.4531
Epoch 298/5000
1/1 [==============================] - 0s 28ms/step - loss: 577992.5625 - val_loss: 28889.0449
Epoch 299/5000
1/1 [==============================] - 0s 27ms/step - loss: 640836.6875 - val_loss: 301380.6250
Epoch 300/5000
1/1 [==============================] - 0s 26ms/step - loss: 610222.8750 - val_loss: 350029.3750
Epoc

...

1/1 [==============================] - 0s 27ms/step - loss: 482255.1562 - val_loss: 58198.7344
Epoch 1552/5000
1/1 [==============================] - 0s 27ms/step - loss: 329546.3438 - val_loss: 7980.2119
Epoch 1553/5000
1/1 [==============================] - 0s 29ms/step - loss: 278353.1562 - val_loss: 39.4173
Epoch 1554/5000
1/1 [==============================] - 0s 26ms/step - loss: 277590.9688 - val_loss: 13424.7529
Epoch 1555/5000
1/1 [==============================] - 0s 28ms/step - loss: 167199.3750 - val_loss: 15855.8271
Epoch 1556/5000
1/1 [==============================] - 0s 27ms/step - loss: 168460.6250 - val_loss: 6688.0132
Epoch 1557/5000
1/1 [==============================] - 0s 25ms/step - loss: 377049.7500 - val_loss: 7010.3008
Epoch 1558/5000
1/1 [==============================] - 0s 28ms/step - loss: 332693.9062 - val_loss: 71441.8750
Epoch 1559/5000
1/1 [==============================] - 0s 33ms/step - loss: 148269.0781 - val_loss: 19.9261
Epoch 1560/5000
1/1 [=====

1/1 [==============================] - 0s 25ms/step - loss: 740758.8125 - val_loss: 40324.4844
Epoch 1626/5000
1/1 [==============================] - 0s 27ms/step - loss: 480622.2188 - val_loss: 34564.0391
Epoch 1627/5000
1/1 [==============================] - 0s 27ms/step - loss: 248125.0000 - val_loss: 7874.1357
Epoch 1628/5000
1/1 [==============================] - 0s 25ms/step - loss: 376195.5938 - val_loss: 82224.4453
Epoch 1629/5000
1/1 [==============================] - 0s 37ms/step - loss: 351234.0938 - val_loss: 48779.7266
Epoch 1630/5000
1/1 [==============================] - 0s 34ms/step - loss: 208088.3906 - val_loss: 354.9029
Epoch 1631/5000
1/1 [==============================] - 0s 29ms/step - loss: 475996.8438 - val_loss: 48792.6680
Epoch 1632/5000
1/1 [==============================] - 0s 26ms/step - loss: 376824.2188 - val_loss: 54098.4883
Epoch 1633/5000
1/1 [==============================] - 0s 27ms/step - loss: 307470.2188 - val_loss: 39896.1602
Epoch 1634/5000
1/1 

1/1 [==============================] - 0s 26ms/step - loss: 95219.0703 - val_loss: 1486.7651
Epoch 1700/5000
1/1 [==============================] - 0s 32ms/step - loss: 112707.6172 - val_loss: 43384.3320
Epoch 1701/5000
1/1 [==============================] - 0s 32ms/step - loss: 326600.6250 - val_loss: 22081.8438
Epoch 1702/5000
1/1 [==============================] - 0s 33ms/step - loss: 315341.7188 - val_loss: 14666.5342
Epoch 1703/5000
1/1 [==============================] - 0s 34ms/step - loss: 300086.1875 - val_loss: 1098.4952
Epoch 1704/5000
1/1 [==============================] - 0s 31ms/step - loss: 201163.6875 - val_loss: 1912.8665
Epoch 1705/5000
1/1 [==============================] - 0s 30ms/step - loss: 221680.9219 - val_loss: 4473.9639
Epoch 1706/5000
1/1 [==============================] - 0s 30ms/step - loss: 592125.5625 - val_loss: 2723.7390
Epoch 1707/5000
1/1 [==============================] - 0s 28ms/step - loss: 297646.6250 - val_loss: 33960.6602
Epoch 1708/5000
1/1 [==

1/1 [==============================] - 0s 26ms/step - loss: 193481.0469 - val_loss: 21098.4141
Epoch 1774/5000
1/1 [==============================] - 0s 27ms/step - loss: 101958.3516 - val_loss: 7007.3574
Epoch 1775/5000
1/1 [==============================] - 0s 27ms/step - loss: 132221.1094 - val_loss: 10926.5781
Epoch 1776/5000
1/1 [==============================] - 0s 28ms/step - loss: 680918.0000 - val_loss: 22150.1016
Epoch 1777/5000
1/1 [==============================] - 0s 29ms/step - loss: 154402.6719 - val_loss: 2515.4536
Epoch 1778/5000
1/1 [==============================] - 0s 29ms/step - loss: 166565.6250 - val_loss: 5038.0884
Epoch 1779/5000
1/1 [==============================] - 0s 32ms/step - loss: 472728.5000 - val_loss: 167507.9375
Epoch 1780/5000
1/1 [==============================] - 0s 30ms/step - loss: 379175.0938 - val_loss: 59942.7227
Epoch 1781/5000
1/1 [==============================] - 0s 25ms/step - loss: 138016.4531 - val_loss: 2753.3796
Epoch 1782/5000
1/1 

1/1 [==============================] - 0s 26ms/step - loss: 609292.2500 - val_loss: 127772.7344
Epoch 1848/5000
1/1 [==============================] - 0s 27ms/step - loss: 294613.0625 - val_loss: 10285.8027
Epoch 1849/5000
1/1 [==============================] - 0s 24ms/step - loss: 298581.0312 - val_loss: 34950.3750
Epoch 1850/5000
1/1 [==============================] - 0s 26ms/step - loss: 213302.2969 - val_loss: 11293.2129
Epoch 1851/5000
1/1 [==============================] - 0s 26ms/step - loss: 314830.4688 - val_loss: 59478.3516
Epoch 1852/5000
1/1 [==============================] - 0s 27ms/step - loss: 181748.0000 - val_loss: 31555.4980
Epoch 1853/5000
1/1 [==============================] - 0s 25ms/step - loss: 169488.8594 - val_loss: 26534.6289
Epoch 1854/5000
1/1 [==============================] - 0s 26ms/step - loss: 294538.4688 - val_loss: 49118.9375
Epoch 1855/5000
1/1 [==============================] - 0s 25ms/step - loss: 64617.0781 - val_loss: 6509.8423
Epoch 1856/5000
1/

In [30]:
prediction = trained_model.predict(test_input)
print(f"{test_target.values}(test_target) - {prediction}(prediction) = {mean_absolute_error(test_target, prediction)}")

1/1 [==============================] - 0s 37ms/step
[12113.838](test_target) - [[11363.082]](prediction) = 750.7559687499997


In [31]:
def model_time_series_mae_tester(data, window, seed):
    if window > 2:

        # Copia e organiza data geral
        predictions = []
        targets = []
        data_copy = data.copy()
        data_copy[subject] = data[subject].shift(-1)
        data_copy = data_copy.iloc[:-1,1:]
        
        input_df = data_copy.iloc[:, :-1]
        target_df = data_copy.iloc[:, -1:]

        for i in range(window, len(data_copy) + 1):

            train_input = input_df.iloc[:i - 1]
            train_input = (train_input - np.mean(train_input, axis=0)) / np.std(train_input, axis=0)
            print("train_input:")
            display(train_input)

            train_target = target_df.iloc[:i - 1]
            print("train_target:")
            display(train_target)

            test_input = input_df.iloc[:i]
            test_input = ((test_input - np.mean(test_input, axis=0)) / np.std(test_input, axis=0)).iloc[-1:]
            print("test_input:")
            display(test_input)

            test_target = target_df.iloc[i - 1:i]
            print("test_target:")
            display(test_target)
            
            model, hitory = neural_network_model(train_input, train_target, want_verbose=0, seed=seed)

            prediction = model.predict(test_input)

            current_error = mean_absolute_error(test_target, prediction)
            print(f"Error: {current_error}\n\n")

            predictions.append(prediction[0].item())
            targets.append(test_target.iloc[0, 0])

        return predictions, targets

In [32]:
predictions, targets = model_time_series_mae_tester(data.iloc[:-1,:], 4, winner_seed)
predictions

train_input:


,São Paulo - IDH Longevitude,São Paulo - IDH,São Paulo - PIB - Estadual,São Paulo - PIB - Construção Civil,São Paulo - PIB - Per Capita,São Paulo - PIB - Preços de Mercado,Contratação Comercial (Bi R$),Contratação Home equity (Bi R$),Contratação Livre (Bi R$),Contratação FGTS (Bi R$),...,Precipitation (mm/day),Atmospheric Pressure(mB),Global Radiation - Daylight (Kj/m²),Air Temperature - Dry Bulb (°C),Dew Point Temperature (°C),Air Relative Humidity (%),Wind Gust (m/s),São Paulo - Desemprego,São Paulo - IDH Renda,São Paulo - IDH Educacao
0,-1.396262,-1.413483,-1.301446,1.260406,1.320662,1.321243,1.165029,1.396038,1.335266,-1.330895,...,-1.351896,-1.190913,-1.062801,-1.376940,-0.651693,1.224451,0.665880,-1.414014,1.41371,-1.396073
1,0.892654,0.667379,0.171476,-0.074746,-0.222278,-0.223887,0.111769,-0.502296,-0.264151,0.251276,...,0.316387,1.255983,-0.276585,0.967804,1.412802,0.000587,0.747547,0.686415,-0.73953,0.502499
2,0.503608,0.746104,1.129971,-1.185660,-1.098383,-1.097356,-1.276798,-0.893743,-1.071116,1.079619,...,1.035508,-0.065070,1.339386,0.409135,-0.761109,-1.225038,-1.413427,0.727598,-0.67418,0.893574


train_target:


,São Paulo - Consumo de Cimento (t)
0,11767.653429
1,9811.084286
2,9918.501000


test_input:


...

,São Paulo - IDH Longevitude,São Paulo - IDH,São Paulo - PIB - Estadual,São Paulo - PIB - Construção Civil,São Paulo - PIB - Per Capita,São Paulo - PIB - Preços de Mercado,Contratação Comercial (Bi R$),Contratação Home equity (Bi R$),Contratação Livre (Bi R$),Contratação FGTS (Bi R$),...,Precipitation (mm/day),Atmospheric Pressure(mB),Global Radiation - Daylight (Kj/m²),Air Temperature - Dry Bulb (°C),Dew Point Temperature (°C),Air Relative Humidity (%),Wind Gust (m/s),São Paulo - Desemprego,São Paulo - IDH Renda,São Paulo - IDH Educacao
6,-1.610507,-1.756374,-1.86678,-0.16282,-1.237276,-1.93551,1.16956,2.035192,0.985175,-0.705937,...,-2.449464,-2.342406,0.271094,-1.394884,-2.138289,-1.648541,-1.835018,-1.412546,-0.605246,0.242895


test_target:


,São Paulo - Consumo de Cimento (t)
6,12113.838


Epoch 1914: early stopping
1/1 [==============================] - 0s 33ms/step
Error: 750.7559687499997




[9917.2099609375, 10515.19921875, 10713.7529296875, 11363.08203125]

In [33]:
display(targets)
display(predictions)

[10001.077, 10458.276, 11181.809, 12113.838]

[9917.2099609375, 10515.19921875, 10713.7529296875, 11363.08203125]

In [34]:
mae = mean_absolute_error(predictions, targets)
mae

339.9005742187496

In [35]:
porcentage = mae/np.mean(targets)
porcentage

0.031073072720260508